In [ ]:
import pandas as pd
import json
from openai import OpenAI
import os

/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.getenv("DEEPSEEK_API_KEY")
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

In [45]:
def call_model(content):
    messages = [{"role": "system", "content": 
                 "You are an AI assistant who will assess whether a given citation text is used as a background, result or method section in a scientific paper."
                 "You will be given a json object containing some metadata, where the field \"string\" contains the citation text, "
                 "and you will need to classify the text as one of [background, result, method]."
                 "You will also need to provide a reasoning for your classification."
                 "Example output would look like: {\"classification\": \"result\", \"reasoning\": \"This is the reasoning\"}"
                 "You will be given 10 of such json objects to classify, and you are strictly required to return all 10 classifications with reasonings, separated by a new line."
                 "There should not be any extra new line characters within each of the json classification"
                 
    }, 
    {"role": "user", "content": f"{content}"}]
    response = client.chat.completions.create( model="deepseek-reasoner", messages=messages)
    print(response)
    try:
        if response.choices[0].message.content:
            #print(response["generated_text"][2]["content"])
            content = response.choices[0].message.content.split("\n")
            result = []
            for res in content:
                print(res)
                start = res.find("{")
                end = res.rfind("}") + 1
                json_str = res[start:end]
                answers = json.loads(json_str)
                classification = answers["classification"]
                reasoning = answers["reasoning"]
                result.append([classification, reasoning])
            return result
        else:
            return "Invalid", "No reasoning provided"
    except Exception as e:
        print(e)
        return "Invalid", "No reasoning provided"

In [4]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    print(set(data["label2"]))
    data = data.drop(columns=["label", "label_confidence", "label2_confidence", "label2"])
    return data

In [5]:
data = preprocess("./data/train.jsonl")

{'supportive', 'cant_determine', nan, 'not_supportive'}


In [ ]:
labels = []
reasonings = []
raw_output = []
ids = []

for i in range(1190, len(data), 10):
    if i % 50 == 0:
        print(i)
    current_data = [data.iloc[i + x].to_json() for x in range(min(10, len(data) - i))]
    for x in range(min(10, len(data) - i)):
        ids.append(data.iloc[i + x].unique_id)
    results = call_model(current_data)
    for result in results:
        labels.append(result[0])
        reasonings.append(result[1])

ChatCompletion(id='b7d6ae0c-a4d6-43d7-9529-b73a79f2ef76', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"classification": "result", "reasoning": "The citation describes a specific finding from the authors\' own study (\'we found that...\'), which is typical in the Discussion section where results are interpreted."}\n{"classification": "method", "reasoning": "The citation refers to the use of an instrument as a \'gold standard\' in validation studies, indicating methodological context or tool selection common in the Method section."}\n{"classification": "method", "reasoning": "The text describes experimental procedures (diet assignments, activity wheels) and cites a prior study for methodological details, typical of the Methods/Experimental Design section."}\n{"classification": "background", "reasoning": "The citation summarizes previously reported enzyme characteristics to establish context for the current investigation, common in

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("deepseek_third.csv")